In [1]:
import pandas as pd
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
import os


image_directory_correct = "./correct/"
image_directory_incorrect = "./incorrect/"
image_directory_without = "./without/"


output = './images/'


In [2]:
def getJSON(filePathandName):
    with open(filePathandName,'r') as f:
        return json.load(f)

In [3]:
cvNet = cv2.dnn.readNetFromCaffe('../caffe-face-detector-opencv-pretrained-model/architecture.txt','../caffe-face-detector-opencv-pretrained-model/weights.caffemodel')

In [4]:
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))

In [5]:
import sys
sys.path.append("/home/saraiva/Desktop/University/SI/AP/TP/yoloface")

from face_detector import YoloDetector


In [6]:
%matplotlib inline


def predYOLOGraphic(test_images):

    gamma = 2.0
    fig2 = plt.figure(figsize = (14,14))
    rows = 3
    cols = 2
    axes = []
    assign = {0:"mask", 1:"no_mask" , 2:"incorrect"}

    yolo = YoloDetector(target_size=720, device="cuda:0", min_face=90)
    for j,im in enumerate(test_images):
        image =  cv2.imread(os.path.join(image_directory_correct,im),1)
        image =  adjust_gamma(image, gamma=gamma)
        image = cv2.resize(image, (992,720))
        bboxes,_ = yolo.predict(image)
        
        
        if len(bboxes[0]) != 1:
            
            try:
                
                startX, startY, endX, endY = bboxes[0][0][0], bboxes[0][0][1],bboxes[0][0][2],bboxes[0][0][3]
                print(startX,startY,endX,endY)
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
                cv2.putText(image,assign[2] , (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (36,255,12), 2)
            
            except Exception as e:
                print(e)
            
        axes.append(fig2.add_subplot(rows, cols, j+1))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    

In [7]:
def predYOLO(img,image_directory):

    res = []
    gamma = 2.0
    yolo = YoloDetector(target_size=720, device="cuda:0", min_face=90)
    
    image =  cv2.imread(os.path.join(image_directory,img),1)
    image =  adjust_gamma(image, gamma=gamma)
    image = cv2.resize(image, (992,720))
    cv2.imwrite(output + img,image)
    bboxes,_ = yolo.predict(image)
    
    for i in range(0, len(bboxes[0])):
        startX, startY, endX, endY = bboxes[0][i][0], bboxes[0][i][1],bboxes[0][i][2],bboxes[0][i][3]
        res.append([startX, startY, endX, endY])

    return res




In [8]:


def createDf(directories):

    df = pd.DataFrame(columns = ['name','x1', 'x2','y1','y2','classname'])

    for d,label in directories:

        files = os.listdir(d)

        for f in files:
            bbs = predYOLO(f,d)
            print(bbs)
            for bb in bbs:
                    startX, startY, endX, endY = bb
                    df = df.append({'name' : f, 'x1' : startX, 'x2' : endX, 'y1':startY, 'y2':endY, 'classname':label},ignore_index = True)


    df.to_csv('test.csv')

directories = [[image_directory_incorrect,2],[image_directory_correct,0],[image_directory_without,1]]

createDf(directories)



cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[355, 235, 508, 412]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[392, 136, 711, 512]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[264, 37, 588, 316]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[403, 0, 831, 474]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[273, 107, 466, 351], [484, 142, 567, 252], [652, 273, 777, 427]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[307, 157, 580, 478]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[558, 113, 748, 353]]
cuda:0
/home/saraiva/Desktop/University/SI/AP/TP/yoloface/weights/yolov5n_state_dict.pt
[[3

KeyboardInterrupt: 